# Proyecto UPRA — Limpieza y Transformación de Datos

En este Notebook se documenta el proceso de limpieza, transformación y preparación
de la base de datos de insumos provenientes de UPRA, con el fin de obtener un dataset
ordenado y apto para análisis en Power BI.

---

## Paso 1: Importación de librerías
En este paso importamos la librería `pandas`, que nos permitirá cargar y manipular la
base de datos en formato CSV.

In [2]:
 import pandas as pd

## Paso 2: Carga del archivo CSV

En este paso cargamos la base de datos original (`insumos.csv`) en un DataFrame de pandas.
Este DataFrame será la estructura principal con la que realizaremos la limpieza,
transformación y reestructuración de los datos.

Mostramos las primeras filas para verificar que el archivo se cargó correctamente.

In [4]:
df = pd.read_csv("insumos.csv")
df.head()

,Fecha,Indice_total,Total_Fertilizantes,Total_Plaguicidas,Total_Otros,Total_Simples,Total_Compuestos,Total_Herbicidas,Total_Fungicidas,Total_Insecticidas,...,Metomil,Tiametoxam + lambdacihalotrina,Abamectina,Imidacloprid,Profenofos + cipermetrina,Cipermetrina,Profenofos,Total_Coadyuvantes,Total_Reguladores,Total_Molusquicidas
0,09/2025,147.05,165.71,117.63,129.30,164.08,166.73,119.00,112.61,117.15,...,121.46,80.13,98.74,86.09,125.62,120.52,141.40,129.82,125.63,179.14
1,08/2025,145.32,162.85,117.73,129.23,160.88,164.16,119.19,112.70,117.05,...,120.12,81.08,99.58,86.37,125.46,120.61,138.67,129.84,125.29,179.05
2,07/2025,146.43,164.22,118.43,128.96,159.32,167.92,120.50,112.92,116.74,...,119.41,80.76,99.18,86.29,125.18,121.20,138.79,129.41,125.37,186.77
3,06/2025,146.35,163.54,119.33,128.56,156.94,168.63,122.06,112.96,117.04,...,119.23,81.25,99.10,86.60,125.04,123.54,138.28,128.73,125.86,186.38
4,05/2025,145.31,161.79,119.43,128.26,156.82,165.55,122.00,113.07,117.56,...,119.28,81.61,99.43,89.89,124.53,122.89,138.57,128.76,124.50,185.09


## PASO: Transformación de la columna `fecha`

La columna `fecha` viene en formato texto ("mm/YYYY"), por lo que Power BI no la reconoce como una fecha válida.

Para poder usar filtros temporales, ordenar correctamente los meses y crear jerarquías (Año–Mes), convertimos este texto en una fecha real con `pd.to_datetime()` y extraemos:

- Año (`año`)
- Mes (`mes`)
- Nombre del mes (`mes_nombre`)

Además, la fecha se interpretará como el **primer día del mes**, lo cual es adecuado para datos agregados mensuales.


In [7]:
# Convertir la fecha "mm/YYYY" a un objeto datetime real

# Renombrar columna
df = df.rename(columns={"Fecha": "fecha"})

# Convertir mm/YYYY a fecha real
df["fecha"] = pd.to_datetime(df["fecha"], format="%m/%Y")

# Extraer año y mes
df["año"] = df["fecha"].dt.year
df["mes"] = df["fecha"].dt.month

# Diccionario de meses en español
meses_es = {
    1: "Enero", 2: "Febrero", 3: "Marzo",
    4: "Abril", 5: "Mayo", 6: "Junio",
    7: "Julio", 8: "Agosto", 9: "Septiembre",
    10: "Octubre", 11: "Noviembre", 12: "Diciembre"
}

# Asignar nombre del mes en español
df["mes_nombre"] = df["mes"].map(meses_es)


df.head()


,fecha,Indice_total,Total_Fertilizantes,Total_Plaguicidas,Total_Otros,Total_Simples,Total_Compuestos,Total_Herbicidas,Total_Fungicidas,Total_Insecticidas,...,Imidacloprid,Profenofos + cipermetrina,Cipermetrina,Profenofos,Total_Coadyuvantes,Total_Reguladores,Total_Molusquicidas,año,mes,mes_nombre
0,2025-09-01,147.05,165.71,117.63,129.30,164.08,166.73,119.00,112.61,117.15,...,86.09,125.62,120.52,141.40,129.82,125.63,179.14,2025,9,Septiembre
1,2025-08-01,145.32,162.85,117.73,129.23,160.88,164.16,119.19,112.70,117.05,...,86.37,125.46,120.61,138.67,129.84,125.29,179.05,2025,8,Agosto
2,2025-07-01,146.43,164.22,118.43,128.96,159.32,167.92,120.50,112.92,116.74,...,86.29,125.18,121.20,138.79,129.41,125.37,186.77,2025,7,Julio
3,2025-06-01,146.35,163.54,119.33,128.56,156.94,168.63,122.06,112.96,117.04,...,86.60,125.04,123.54,138.28,128.73,125.86,186.38,2025,6,Junio
4,2025-05-01,145.31,161.79,119.43,128.26,156.82,165.55,122.00,113.07,117.56,...,89.89,124.53,122.89,138.57,128.76,124.50,185.09,2025,5,Mayo


### Verificación previa: columnas que contienen valores cero

Antes de reemplazar los ceros por valores `NaN`, es fundamental verificar si realmente existen ceros en el dataset.  
Esto nos permite:

- Confirmar que la transformación es necesaria.  
- Identificar qué columnas contienen ceros.  
- Evitar modificar columnas donde el cero sí es un valor válido.  

El siguiente código genera una tabla que muestra cuántos ceros tiene cada columna del DataFrame.


In [8]:
# Contar cuántos ceros hay en cada columna
cantidad_ceros = (df == 0).sum()

# Mostrar solo columnas que sí tienen ceros
cantidad_ceros[cantidad_ceros > 0]


Clorpirifos            21
Fipronil               13
Total_Coadyuvantes     25
Total_Reguladores      25
Total_Molusquicidas    25
dtype: int64

## Verificación y análisis de columnas con valores cero

Antes de reemplazar ceros por valores `NaN`, realizamos una verificación para identificar en qué columnas del dataset existen valores igual a cero.  
El análisis arrojó los siguientes resultados:
Clorpirifos 21
Fipronil 13
Total_Coadyuvantes 25
Total_Reguladores 25
Total_Molusquicidas 25


---

### Interpretación de resultados

Las columnas encontradas pueden dividirse en dos grupos según el significado del valor cero:

---

### **1. Ingredientes activos donde 0 significa “no reportado”**

- **Clorpirifos** — 21 ceros  
- **Fipronil** — 13 ceros  

En estos casos, el valor cero **no representa un precio real**, sino la **ausencia de reporte** a partir de ciertos años (por ejemplo, 2024).  
Por esta razón, es necesario **convertir los ceros en valores `NaN`** para evitar interpretaciones incorrectas en Power BI.

---

###  **2. Columnas de totales donde 0 sí es un valor válido**

- **Total_Coadyuvantes** — 25 ceros  
- **Total_Reguladores** — 25 ceros  
- **Total_Molusquicidas** — 25 ceros  

Estas columnas representan **totales agregados por categoría**, por lo que un valor cero es completamente válido (por ejemplo, meses donde no hubo reportes de esa categoría).  
En estos casos, **no se deben reemplazar los ceros por `NaN`**.


In [9]:
import numpy as np

# Columnas donde el cero SÍ significa "no reportado"
columnas_convertir = ["Clorpirifos", "Fipronil"]

# Reemplazar ceros por NaN en esas columnas
df[columnas_convertir] = df[columnas_convertir].replace(0, np.nan)


## Verificamos que los ceros fueron remplazados

In [10]:
# Contar cuántos ceros hay en cada columna
cantidad_ceros = (df == 0).sum()

# Mostrar solo columnas que sí tienen ceros
cantidad_ceros[cantidad_ceros > 0]

Total_Coadyuvantes     25
Total_Reguladores      25
Total_Molusquicidas    25
dtype: int64

## verificamos nombres de las columnas

In [11]:
df.columns

Index(['fecha', 'Indice_total', 'Total_Fertilizantes', 'Total_Plaguicidas',
       'Total_Otros', 'Total_Simples', 'Total_Compuestos', 'Total_Herbicidas',
       'Total_Fungicidas', 'Total_Insecticidas', 'Urea 46%', 'Urea sulfato',
       'DAP 18-46', 'KCL 0-0-60', 'SAM', '15-15-15', '25-4-24', '17-6-18-2',
       '18-18-18', '31-8-8', '12-24-12', '13-26-6', '15-4-23', '10-20-30',
       '28-4-0-6', 'Glifosato', 'Paraquat', 'Propanil', '2,4-D + picloram',
       '2,4-D', 'Aminopiralid + 2,4-D', 'Diuron', 'Glufosinato de amonio',
       'Picloram', 'Oxadiazon', 'Metsulfuron metil', 'Pendimetalin',
       'Clorotalonil', 'Difenoconazol', 'Mancozeb', 'Mancozeb + cimoxanil',
       'Azoxistrobin + difenoconazol', 'Dimetomorf',
       'Tebuconazol + trifloxistrobin', 'Propineb + fluopicolide',
       'Mancozeb + metalaxil-m', 'Clorpirifos', 'Fipronil', 'Metomil',
       'Tiametoxam + lambdacihalotrina', 'Abamectina', 'Imidacloprid',
       'Profenofos + cipermetrina', 'Cipermetrina', 'Profe

### Clasificación de insumos por categoría

Para realizar análisis avanzados (tendencias por familia, variaciones por grupo, correlaciones internas, etc.) es importante clasificar cada insumo dentro de una categoría general.  
El dataset original incluye 58 columnas de diferentes tipos de productos (fertilizantes, herbicidas, fungicidas, insecticidas, coadyuvantes y más).

Para esto se crea un diccionario donde cada insumo se asigna a su categoría correspondiente.  
Esta clasificación será utilizada más adelante cuando se realice el proceso de **despivotado** (wide → long), lo cual permitirá:

- Hacer análisis comparativos entre categorías  
- Construir visualizaciones más claras en Power BI  
- Reducir complejidad en el modelo de datos  
- Aplicar filtros dinámicos por familia de insumos  

A continuación se define el diccionario de categorías basado en los nombres exactos de las columnas del dataset:


In [15]:
categorias = {
    # ---------------------------
    # Fertilizantes simples
    # ---------------------------
    "Urea 46%": "Fertilizante simple",
    "Urea sulfato": "Fertilizante simple",
    "DAP 18-46": "Fertilizante simple",
    "KCL 0-0-60": "Fertilizante simple",
    "SAM": "Fertilizante simple",

    # ---------------------------
    # Fertilizantes compuestos
    # ---------------------------
    "15-15-15": "Fertilizante compuesto",
    "25-4-24": "Fertilizante compuesto",
    "17-6-18-2": "Fertilizante compuesto",
    "18-18-18": "Fertilizante compuesto",
    "31-8-8": "Fertilizante compuesto",
    "12-24-12": "Fertilizante compuesto",
    "13-26-6": "Fertilizante compuesto",
    "15-4-23": "Fertilizante compuesto",
    "10-20-30": "Fertilizante compuesto",
    "28-4-0-6": "Fertilizante compuesto",

    # ---------------------------
    # Herbicidas
    # ---------------------------
    "Glifosato": "Herbicida",
    "Paraquat": "Herbicida",
    "Propanil": "Herbicida",
    "2,4-D + picloram": "Herbicida",
    "2,4-D": "Herbicida",
    "Aminopiralid + 2,4-D": "Herbicida",
    "Diuron": "Herbicida",
    "Glufosinato de amonio": "Herbicida",
    "Picloram": "Herbicida",
    "Oxadiazon": "Herbicida",
    "Metsulfuron metil": "Herbicida",
    "Pendimetalin": "Herbicida",

    # ---------------------------
    # Fungicidas
    # ---------------------------
    "Clorotalonil": "Fungicida",
    "Difenoconazol": "Fungicida",
    "Mancozeb": "Fungicida",
    "Mancozeb + cimoxanil": "Fungicida",
    "Azoxistrobin + difenoconazol": "Fungicida",
    "Dimetomorf": "Fungicida",
    "Tebuconazol + trifloxistrobin": "Fungicida",
    "Propineb + fluopicolide": "Fungicida",
    "Mancozeb + metalaxil-m": "Fungicida",

    # ---------------------------
    # Insecticidas
    # ---------------------------
    "Clorpirifos": "Insecticida",
    "Fipronil": "Insecticida",
    "Metomil": "Insecticida",
    "Tiametoxam + lambdacihalotrina": "Insecticida",
    "Abamectina": "Insecticida",
    "Imidacloprid": "Insecticida",
    "Profenofos + cipermetrina": "Insecticida",
    "Cipermetrina": "Insecticida",
    "Profenofos": "Insecticida",

    # ---------------------------
    # Totales (no se despivotan)
    # ---------------------------
    "Indice_total": "Total",
    "Total_Fertilizantes": "Total",
    "Total_Plaguicidas": "Total",
    "Total_Otros": "Total",
    "Total_Simples": "Total",
    "Total_Compuestos": "Total",
    "Total_Herbicidas": "Total",
    "Total_Fungicidas": "Total",
    "Total_Insecticidas": "Total",
    "Total_Coadyuvantes": "Total",
    "Total_Reguladores": "Total",
    "Total_Molusquicidas": "Total",
}


## Clasificación de insumos por categoría

Para realizar un análisis más claro y permitir una futura reestructuración del dataset (como el proceso de *despivotar* columnas), fue necesario clasificar cada insumo según su categoría técnica.

El dataset original incluye una variedad de insumos agrícolas agrupados en grandes categorías como fertilizantes, herbicidas, fungicidas e insecticidas. Sin embargo, esta clasificación no viene explícita en el archivo CSV, sino que debe reconstruirse a partir del conocimiento de los productos y su posición dentro de la tabla de reportes.

### ¿Por qué es importante clasificar los insumos?

- Permite **despivotar** la tabla correctamente (producto → fila, precio → valor).
- Facilita filtrar y analizar por categorías en Power BI.
- Evita comparar insumos que pertenecen a grupos completamente diferentes.
- Permite construir indicadores por categoría (por ejemplo, análisis solo de fertilizantes compuestos o solo de herbicidas).

---

## Criterio de clasificación aplicado

La clasificación de cada insumo se realizó teniendo en cuenta:

### 1. **La estructura del dataset original**
El archivo reporta totales por categoría:

- `Total_Simples`
- `Total_Compuestos`
- `Total_Herbicidas`
- `Total_Fungicidas`
- `Total_Insecticidas`
- `Total_Coadyuvantes`
- `Total_Reguladores`
- `Total_Molusquicidas`

A partir de esta estructura se puede identificar que todos los insumos individuales listados bajo estas secciones pertenecen a esas categorías.

---

### 2. **La función agronómica y química de cada insumo**

- **Fertilizantes simples**  
  Son nutrientes con una sola fuente principal (ej. Urea, DAP, SAM, KCL).  
  Su composición química es simple y no corresponde a mezclas industriales complejas.

- **Fertilizantes compuestos (NPK)**  
  Son formulaciones industriales con proporciones específicas como 15-15-15, 25-4-24, 12-24-12, etc.  
  Todas estas mezclas pertenecen a la categoría de fertilizantes compuestos.

- **Herbicidas**  
  Insumos destinados al control de malezas. Aquí aparecen principios activos tradicionales como Glifosato, Paraquat, Picloram, Diuron, Metsulfuron metil, entre otros.

- **Fungicidas**  
  Productos enfocados en el control de hongos. Se identifican principios activos típicos como Mancozeb, Clorotalonil, Difenoconazol, Dimetomorf, etc.

- **Insecticidas**  
  Insumos utilizados contra insectos, como Clorpirifos, Fipronil, Abamectina, Imidacloprid, Metomil, etc.

- **Totales**  
  Son indicadores agregados que resumen los índices de precios de cada grupo.  
  **No representan un insumo individual** y por lo tanto no deben clasificarse como tales ni incluirse durante el proceso de despivote.


## 4. Transformación de la base de datos a formato largo (Unpivot)

Después de revisar y limpiar la estructura original, la base seguía estando en **formato ancho**, es decir, cada insumo estaba representado como una columna independiente (más de 50 columnas diferentes).  
Este formato es poco adecuado para análisis en Power BI o para aplicar técnicas de análisis estadístico, ya que dificulta:

- Agrupar por categorías,
- Comparar insumos entre sí,
- Crear visualizaciones dinámicas,
- Realizar análisis temporales por tipo de insumo.

### Objetivo
Convertir la tabla de formato ancho a formato **largo** (long format), donde cada fila representa un insumo específico en una fecha determinada.

---

## 4.1. Identificación de columnas fijas e insumos

Primero definimos:

- **Columnas de identificación (`id_vars`)** → Son las que no deben transformarse:  
  - `fecha`  
  - `año`  
  - `mes`  
  - `mes_nombre`

- **Columnas de valor (`value_vars`)** → Todas las demás columnas, que corresponden a cada insumo.

Estas columnas se detectan dinámicamente para evitar errores y mantener flexibilidad si la base cambia.

---

## 4.2. Proceso de despivotar con `pandas.melt()`

Utilizamos la función `melt()` de pandas para transformar la base:

- Cada insumo pasa de estar en una columna → a convertirse en una fila.
- Los valores numéricos se almacenan en una columna llamada **`indice`**.
- El nombre del insumo se almacena en una columna llamada **`insumo`**.

Resultado esperado (formato largo):

| fecha       | año | mes | insumo     | indice | categoría |
|-------------|-----|-----|------------|--------|-----------|
| 2024-01-01  | 2024| 1   | urea_46    | 207.71 | Fertilizante simple |
| 2024-01-01  | 2024| 1   | glifosato  | 146.47 | Herbicida |

Este formato facilita todas las etapas del análisis posterior.

---

## 4.3. Asignación de categorías a cada insumo

Previamente se construyó un diccionario llamado `categorias`, donde se definió la categoría correspondiente a cada insumo según su función:

- Fertilizantes simples  
- Fertilizantes compuestos  
- Herbicidas  
- Fungicidas  
- Insecticidas  
- Coadyuvantes  
- Reguladores  
- Molusquicidas  

Durante la transformación larga, simplemente se añade una nueva columna:

```python
df_long["categoria"] = df_long["insumo"].map(categorias)

Esto permite que cada insumo tenga identificada su familia o tipo, facilitando:

Visualizaciones temáticas,

Comparaciones por grupo,

Análisis por categoría a lo largo del tiempo.

4.4. Resultado final

El resultado es una única tabla estandarizada, limpia y analíticamente útil, con la siguiente estructura:
fecha
año
mes
mes_nombre
insumo
indice
categoria

In [17]:
# 1. Identificamos las columnas fijas
id_cols = ["fecha", "año", "mes", "mes_nombre"]

# 2. Columnas que NO se deben despivotar
columnas_totales = [
    "Indice_total",
    "Total_Fertilizantes",
    "Total_Plaguicidas",
    "Total_Otros",
    "Total_Simples",
    "Total_Compuestos",
    "Total_Herbicidas",
    "Total_Fungicidas",
    "Total_Insecticidas",
    "Total_Coadyuvantes",
    "Total_Reguladores",
    "Total_Molusquicidas"
]

# 3. Columnas que sí se deben despivotar (solo insumos)
valor_cols = [
    col for col in df.columns 
    if col not in id_cols and col not in columnas_totales
]

# 4. Despivotar
df_long = df.melt(
    id_vars=id_cols,
    value_vars=valor_cols,
    var_name="insumo",
    value_name="indice"
)

# 5. Asignar categoría
df_long["categoria"] = df_long["insumo"].map(categorias)


In [21]:
df_long.head()

,fecha,año,mes,mes_nombre,insumo,indice,categoria
0,2025-09-01,2025,9,Septiembre,Urea 46%,171.39,Fertilizante simple
1,2025-08-01,2025,8,Agosto,Urea 46%,165.37,Fertilizante simple
2,2025-07-01,2025,7,Julio,Urea 46%,164.47,Fertilizante simple
3,2025-06-01,2025,6,Junio,Urea 46%,161.29,Fertilizante simple
4,2025-05-01,2025,5,Mayo,Urea 46%,160.69,Fertilizante simple


In [23]:
df_long.shape

(3690, 7)

## Validar que no haya insumos sin categoría

In [24]:
df_long[df_long["categoria"].isna()]["insumo"].unique()


array([], dtype=object)

devuelve un array vacío, está perfecto.

## Revisar valores faltantes (general y específico)
Revisión general:

In [25]:
df_long.isna().sum()


fecha          0
año            0
mes            0
mes_nombre     0
insumo         0
indice        34
categoria      0
dtype: int64

Revisión en columnas críticas:

In [26]:
df_long[["fecha", "insumo", "indice", "categoria"]].isna().sum()


fecha         0
insumo        0
indice       34
categoria     0
dtype: int64

Validar tipos de datos:

In [27]:
df_long.dtypes


fecha         datetime64[ns]
año                    int32
mes                    int32
mes_nombre            object
insumo                object
indice               float64
categoria             object
dtype: object

Revisar que no existan duplicados inesperados:

In [28]:
df_long.duplicated().sum()


np.int64(0)

Ordenar los datos, para que en Power BI todo quede en orden temporal:

In [33]:
df_long = df_long.sort_values(by=["fecha", "categoria", "insumo"])
df_long.head(200)

,fecha,año,mes,mes_nombre,insumo,indice,categoria
1147,2018-12-01,2018,12,Diciembre,10-20-30,90.19,Fertilizante compuesto
901,2018-12-01,2018,12,Diciembre,12-24-12,87.69,Fertilizante compuesto
983,2018-12-01,2018,12,Diciembre,13-26-6,89.86,Fertilizante compuesto
491,2018-12-01,2018,12,Diciembre,15-15-15,89.24,Fertilizante compuesto
1065,2018-12-01,2018,12,Diciembre,15-4-23,83.32,Fertilizante compuesto
...,...,...,...,...,...,...,...
2619,2019-04-01,2019,4,Abril,Azoxistrobin + difenoconazol,91.14,Fungicida
2291,2019-04-01,2019,4,Abril,Clorotalonil,88.63,Fungicida
2373,2019-04-01,2019,4,Abril,Difenoconazol,92.20,Fungicida
2701,2019-04-01,2019,4,Abril,Dimetomorf,87.29,Fungicida


Exportación del dataset limpio a CSV:

In [32]:
df_long.to_csv("insumos_limpio_para_powerbi.csv", index=False, encoding="utf-8")
